In [ ]:
!nvidia-smi

Mon Apr  3 21:28:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# transformers 설치
!pip install -qq transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

# kogpt 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)
_ = model.eval()

# 사람이 프롬프트를 입력하면, 인공지능이 답변하는 프로세스
prompt = '인공지능아, 너는 말을 할 수 있니?'
with torch.no_grad():
  # 사람의 프롬프트를 컴퓨터가 이해할 수 있게 인코드
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)

  # 인공지능이 문장을 생성함 
  # temperature 인자는 모델이 다양한 결과를 내도록 함(0에 가까울수록 매번 같은 결과)
  # max_length는 생성되는 문장의 길이
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)

  # 생성된 문장을 사람이 이해할 수 있게 디코드
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

인공지능아, 너는 말을 할 수 있니? 인공지능은 우리가 생각했던 것보다 훨씬 더 빨리 다가오고 있다. 인공지능은 이미 우리 사회 곳곳에서 우리가 미처 생각지도 못한 사이에 우리의 삶에 깊숙이 침투하고 있다. 인공지능의 역사는 생각보다 길지 않


In [8]:
prompt = '요즘 벚꽃 구경하러 가기 좋은 곳이 어디니'

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
generated

'요즘 벚꽃 구경하러 가기 좋은 곳이 어디니? 라고 묻는다면 고민하지 않고 서울의 벚꽃명소가 될 만한 곳이 있어요. 바로 양재천인데요. 양재천을 따라 벚꽃이 정말 가득하게 펴있거든요. 서울에 살고 있기 때문에 양재천의 벚꽃'

In [9]:
# 책쓰기
prompt = '''제목: ChatGPT 시대 인간의 역할
내용: 사람은 인공지능을 만듭니다. 인공지능은 계속 발전하고 있습니다.
인간은 그런 인공지능을 어떻게 활용해야 할지 고민합니다.
'''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

제목: ChatGPT 시대 인간의 역할
내용: 사람은 인공지능을 만듭니다. 인공지능은 계속 발전하고 있습니다.
인간은 그런 인공지능을 어떻게 활용해야 할지 고민합니다.
인류 역사가 시작된 이래로, 인간은 끊임없이 더 나은 삶을 꿈꾸며 살아왔습니다.
지금도 마찬가지입니다.
우리는 앞으로 어떤 시대가 올지에 관한 이야기를 해보고자 합니다.
Chapter 1. 인공지능과 인간의 경계가 무너지다
Chapter 2. 인공지능의 한계를 뛰어넘다
Chapter 3. 인공지능의 가능성은 무궁무진하다
Chapter 4. 인공지능이 가져올 변화
Chapter 5. 인간이 인공지능을 만들다
Chapter 6. AI, 인간의 삶을 바꾸다
Chapter 7. 인간의 존엄성을 지키다
Chapter 8. 인간과 AI가 공존하기 위한 방법
 Chapter 9. 인공지능을 활용한 일자리 창출
Chapter 10. 인공지능을 활용한 법률의 한계
Chapter 11. AI에서의 윤리문제
인공지능은 우리의 삶을 바꾸고 있습니다.
인공지능을 활용한 법률, 의료서비스, 자율주행차 등 다양한 분야에서 변화가 이뤄지고 있습니다.
인공지능은 단순히 AI 자체로 머무는 것이 아니라, 다른 모든 기술과 함께 우리의 삶에 영향을 미치고 있습니다.
인공지능이 만들어가는 미래를 전망하고, 그 변화 위에서 내가 할 수 있는 일이 무엇인지 찾아보는 시간을 갖고자 합니다.
인공지능에 관해 가장 권위 있는 두 사람을 소개합니다.
먼저, 《타임》에서 선정한 세계에서 가장 영향력 있는 인물 100인 중 한 명인 미치오 카쿠입니다.
세계적 과학저술가 미치오 카쿠는 과학 저술가, 저널리스트, 작가, 기업가로 활동하고 있습니다.
그가 집필한 『마음의 미래』는 인공지능이 가져올 인간의 미래에 대해 다루고 있습니다.
『사이언스』와 『뉴욕타임스』, 『월스트리트저널』, 『이코노미스트』 등 세계 유수 매체에 과학 칼럼을 기고해 왔으며, 지금까지 100권이 넘는 책을 집필하였습니다.
그는 인공지능과 관련하여 "2030년경 인류를 정복할 인

In [10]:
# QA
prompt = '''Cityzen Developer 과정에 AI 전문가 KoGPT님을 모셨습니다. 
Q : 반도체 분야에서 인공지능을 어떻게 활용하면 좋을까요?
A : '''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

Cityzen Developer 과정에 AI 전문가 KoGPT님을 모셨습니다. 
Q: 반도체 분야에서 인공지능을 어떻게 활용하면 좋을까요?
A :  반도체 분야에서 인공지능은 머신러닝과 딥러닝, 인지AI를 활용할 수 있습니다. 머신러닝은 인공지능의 한 분야로 데이터를 분석하고 학습하여 더 나은 성능을 가지는 것을 의미하며, 딥러닝은 머신러닝의 한 단계로 이미지, 음성 및 자연어 처리 분야에 주로 사용됩니다. 인지AI는 머신러닝과 딥러닝의 성과를 사람이 예측, 분석 및 이해할 수 있는 방법을 찾는 것을 의미합니다.
머신러닝은 크게 지도학습과 비지도학습으로 구분할 수 있습니다. 지도학습은 분류, 회귀, 군집 등의 방법이 있으며, 비지도학습은 자연어 처리, 음성 인식, 음성 합성, 이미지 처리, 텍스트 처리 등을 활용할 수 있습니다. 딥러닝은 지도학습과 비지도학습을 모두 포함합니다.
Q: 반도체 분야에서 인공지능을 어떻게 활용하면 좋을까요?
A :  인공지능(AI)은 반도체 분야의 혁신기술입니다. 인공지능은 데이터를 학습하여 더 나은 성능을 가지는 것을 의미하며, 반도체 업계에서는 기술의 한계를 극복하기 위해 머신러닝과 딥러닝을 활용하고 있습니다.
인공지능은 반도체 산업의 구조를 변화시키며, 더 나은 성능을 가진 제품과 서비스를 제공할 수 있습니다. 인공지능이 반도체 산업을 변화시키는 방법과 인공지능이 반도체 산업을 변화시키는 방법은 다릅니다. 인공지능은 반도체 산업에서 일하는 방식, 제품을 만들고 서비스를 제공하는 방식을 모두 변화시킵니다.
Q: 인공지능과 반도체 산업의 구조가 어떻게 다른데요?
A :  인공지능은 기술의 한계를 넘어서도록 만들고, 신기술이 나올 수 있게 합니다. 그러나 인공지능을 활용할 수 있는 기술은 제한적입니다. 머신러닝과 딥러닝을 활용할 수 있는 기술로는 이미지 처리, 음성 인식, 음성 합성, 자연어 처리 등이 있습니다.
머신러닝과 딥러닝의 학습을 위해서는 대량의 이미지, 음성, 텍스트를 학습할 수 있는 고성능 컴퓨팅 시스템이 필요합니다. 

In [16]:
# 뉴스 한줄 요약
prompt = '''[디지털데일리 최민지 기자] 네이버가 700조원 규모 사우디아라비아 초대형 친환경 도시 프로젝트 ‘네옴시티’ 사업 수주전에 뛰어든다. 이미 사우디에 팀네이버 기술력을 소개한 만큼, 네이버는 이번 사업수주를 위해 최선을 다하겠다는 의지를 드러냈다.
강상철 네이버랩스 책임리더는 23일 네이버 1784에서 열린 ‘네이버 테크포럼’ 간담회를 통해 “사우디 네옴시티 건설 초기 단계지만, 미래 스마트시티에서 네이버 솔루션을 활용할 수 있는 부분이 많으니 열심히 해 볼 생각”이라고 말했다. 이동환 네이버랩스 책임리더 또한 “최근 네옴시티가 큰 화제였는데, 네옴시티에 네이버 기술을 활용할 수 있도록 최선을 다하겠다”고 강조했다.
특히, 강상철 책임리더는 지난 사우디 방문 일정을 함께한 네이버 임원진 중 한 명이다. 앞서, 채선주 네이버 대외·ESG 대표는 네이버랩스‧네이버클라우드 주요 임원 및 기술진과 ‘팀네이버’를 꾸려 원희룡 국토교통부 장관 사우디 방문 일정에 동행, 지난 6일 ‘한-사우디 혁신 로드쇼’에 참가했다. 강 책임리더 등은 사우디 현장에서 팀네이버 기술력을 소개했다. 이는 네옴시티 수주를 위한 행보다.
사우디 측은 네이버 기술력에 관심을 드러낸 것으로 전해진다. 실제, 네옴시티 방문 때 네이버는 네이버랩스‧클라우드 기술들을 발표했고, 1:1 상담을 계속 진행했다. 정부 관계자들도 다수 찾았다는 후문이다.
강 책임리더는 “디지털트윈에 대한 질문이 많았다. 어디에 쓸 수 있고, 어느 업체와 협력할 수 있는지 등을 물었다”며 “초기단계지만 네이버 솔루션에 관심을 갖고 있었고, 구체적 내용은 협의해야 한다”고 설명했다.

한줄 요약: '''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

[디지털데일리 최민지 기자] 네이버가 700조원 규모 사우디아라비아 초대형 친환경 도시 프로젝트 ‘네옴시티’ 사업 수주전에 뛰어든다. 이미 사우디에 팀네이버 기술력을 소개한 만큼, 네이버는 이번 사업수주를 위해 최선을 다하겠다는 의지를 드러냈다.
강상철 네이버랩스 책임리더는 23일 네이버 1784에서 열린 ‘네이버 테크포럼’ 간담회를 통해 “사우디 네옴시티 건설 초기 단계지만, 미래 스마트시티에서 네이버 솔루션을 활용할 수 있는 부분이 많으니 열심히 해 볼 생각”이라고 말했다. 이동환 네이버랩스 책임리더 또한 “최근 네옴시티가 큰 화제였는데, 네옴시티에 네이버 기술을 활용할 수 있도록 최선을 다하겠다”고 강조했다.
특히, 강상철 책임리더는 지난 사우디 방문 일정을 함께한 네이버 임원진 중 한 명이다. 앞서, 채선주 네이버 대외·ESG 대표는 네이버랩스‧네이버클라우드 주요 임원 및 기술진과 ‘팀네이버’를 꾸려 원희룡 국토교통부 장관 사우디 방문 일정에 동행, 지난 6일 ‘한-사우디 혁신 로드쇼’에 참가했다. 강 책임리더 등은 사우디 현장에서 팀네이버 기술력을 소개했다. 이는 네옴시티 수주를 위한 행보다.
사우디 측은 네이버 기술력에 관심을 드러낸 것으로 전해진다. 실제, 네옴시티 방문 때 네이버는 네이버랩스‧클라우드 기술들을 발표했고, 1:1 상담을 계속 진행했다. 정부 관계자들도 다수 찾았다는 후문이다.
강 책임리더는 “디지털트윈에 대한 질문이 많았다. 어디에 쓸 수 있고, 어느 업체와 협력할 수 있는지 등을 물었다”며 “초기단계지만 네이버 솔루션에 관심을 갖고 있었고, 구체적 내용은 협의해야 한다”고 설명했다.

한줄 요약:  네이버랩스 중동·동남아에 이어 사우디 정부까지. 네이버는 기술력을 앞세워 네옴시티 사업 수주에 도전한다.
https://www.ddaily.co.kr/news/article.html?no=156590
https://www.edaily.co.kr/news/read?newsId=02827466258661292&mediaCodeNo=257


In [12]:
# 사투리로 변환
prompt = '''
"비도 오는데 어디가서 파전에 막걸리나 한 잔 할까?"
충청도 사투리:
'''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=256)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


"비도 오는데 어디가서 파전에 막걸리나 한 잔 할까?"
충청도 사투리:
"비오는 날에는 막걸리나지. 파전에 막걸리가 딱이라며?"
충청도 사투리:
"비오는 날에는 막걸리지. 파전에 막걸리가 딱이라며?"
충청도 사투리:
"비오는 날에는 막걸리지. 파전에 막걸리가 딱이라며?"
충청도 사투리:
충청도 사투리:
"비 오는 날에는 막걸리지. 파전에 막걸리가 딱이라며?"
충청도 사투리:
충청도 사투리:
"비 오는 날에는 막걸리지. 파전에 막걸리가 딱이라며?"[EOS]


In [15]:
# 영어 번역
prompt = '''
"비도 오는데 어디가서 파전에 막걸리나 한 잔 할까?"
English Translation:
'''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=256)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


"비도 오는데 어디가서 파전에 막걸리나 한 잔 할까?"
English Translation:
"Oh, there's no good to do at all,
just sit on the river, and drink the wine,
but I'm tired on the way,
so I'll buy some ordinary glasses."
"Oh, no, just sit on the river, and drink the wine,
but I'm tired on the way, so I'll buy some ordinary glasses."
일상에 지친 직장인들에게 '그냥 앉아서 하늘을 보며 마시는 한 잔의 술이 얼마나 여유가 있는지'를 느끼게 하고 싶다.
<!--[endif]-->
Dialogue A: Do you remember how we used to sit on the river for a drink?
B: I don't remember, but it was okay to sit on the river,
and drink the wine.
A: Oh, it's so nice to drink the wine.
B: It's okay to drink the wine.
A: I guess it's true


In [21]:
# 챗봇
prompt = '''이루다: 너 어제 어디 갔었어? 전화 했는데 왜 안받았어?
이루나: '''

with torch.no_grad():
    tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=150)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)

이루다: 너 어제 어디 갔었어? 전화 했는데 왜 안받았어?
이루나:  (짜증내며)  왜 그래, 나도 집에 있었어
  (2016. 3. 26. 관찰기록) 위의 예시는 친구와의 갈등이 있는 상황에서 친구와 갈등상황을 해결하려고 시도하는 내용이다.  그러나 연구 참여자의 화를 내기 시작한 것은 갈등상황이 시작된 뒤 약 30분정도 되는 시간이다.  이후 연구 참여자는 친구와의 갈등상황을 해결하기 위해 갈등을 일으킬만한 상황에서는 갈등이 해결되지는 않았지만 자신이 화내지 않은 것에는 만족스러워 하였다.  그러나 이러한


In [24]:
n_conversations = 10

person = '이루다'
prompt = '''이루다: 너 오늘 어디 있었어?
이루나: '''

print(prompt.split('\n')[0])

for i in range(n_conversations):
    with torch.no_grad():
        tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
        gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=32)
        generated = tokenizer.batch_decode(gen_tokens)[0]

        splited = generated.split('\n')
        answer = splited[1]

        print(answer)

        if person == '이루다':
            person = '이루나'
        else:
            person = '이루다'

        prompt = answer + f'\n{person}:'

이루다: 너 오늘 어디 있었어?
이루나:  뭐하느라 늦게 왔어!
이루나:  네.  방금 왔어요
이루다:  응
이루나:  나?
이루다:  어....
이루나: 내가 어..
이루다: 어..
이루나: 우리 뭐하러 온거야?
이루다: 아! 우리..[EOS]
이루나: 너 내가 몇번을 말해. 내가 너 싫다고 했지.
